<a href="https://colab.research.google.com/github/HanlunAI/ExhibitionDemonstration/blob/main/%E5%8F%AF%E4%BB%A5%E7%8E%A9%E5%91%B3%E7%9A%84%E8%88%AA%E9%81%93%E8%BB%8C%E8%B7%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**前言**:地球上的航道也是可以玩味的軌跡，我們先以球面的對稱性處理相關的變分法問題(Calculus of variations) ，同學可以透過 Geogebra 互動素材體驗航道的設計。力所能及的同學更可以使用 Colab 進行更多探索，進而引申至一些後續問題供將來思考。

-- Harold Chao, YK Tai 2022

注意安裝繪製地圖的套件需時頗長，在按下運行鍵/Shift+Enter後請先體驗 Geogebra 互動素材。

In [1]:
!pip install --no-binary shapely shapely --force
!pip install cartopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 200 kB 15.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for shapely: filename=Shapely-1.8.5.post1-cp37-cp37m-linux_x86_64.whl size=669749 sha256=9297d782453b215b7cce9424c7a9bb3fdd2325d69d63c0fe73e720266fe8e9cd
  Stored in directory: /root/.cache/pip/wheels/a3/07/7c/cc50e8ecc795963726466b22ecd395ee1e2a28da87a4cdeb56
Successfully built shapely
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.8.5.post1
    Uninstalling Shapely-1.8.5.post1:
      Successfully uninstalled Shapely-1.8.5.post1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.8 MB 21.4 MB/s 
  Installing build dependencies ... do

[互動素材(點擊打開)](https://www.geogebra.org/classic/mesdgrmm)：以一理想球體作為地球的模型，每座城市為球面上的一點，一航班需要通航於兩座城市之間，問如何尋得最短航道？
*![picture](https://drive.google.com/uc?id=1qX8fh24E-I6CMspzWusgcvEwWzcVSrHR)*

[答(點擊打開)](https://www.geogebra.org/classic/wqwxrd6y)：考慮過三點(城市A, 城市B, 地球中心)平面與球體相交的曲線，即可尋得最短航道--大圓航線。
*![picture](https://drive.google.com/uc?id=1TN-n61f6LWBRpGz4ugnzoCSbm2MQ63RQ)*

[理解(點擊打開)：因為球體的對稱性，以城市 A 為中心的圓圈，代表了由城市 A 出發可以到達特定距離的最遠邊界；同理，以城市 B 為中心的圓圈，也代表了由城市 B 出發可以到達特定距離的最遠邊界。最短航道上的點剛好是以城市 A 的某圈與以城市 B 的某圈的相切點（否則，收窄其中一圈至與另一圈相切，便會產生更短航道），又由於對稱性，該點必處於過三點(城市A，城市B，地球中心)平面之上。於是，我們可以考慮此平面與球體相交的曲線來尋找最短航道，此航道正交於層層包圍圈，正是大圓的一部分，故稱為大圓航線。](https://www.geogebra.org/classic/cjdrtfbb)

*![picture](https://drive.google.com/uc?id=121t0K1DF-VsH3rLKldfmVWGsJp0JSU31)*


In [11]:
#@title Colab 進行更多探索：預備 { run: "auto" }
PackagesInstalledPreviously = "Yes" #@param ["Yes", "No"]

import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

def orthonormalFrame(a,b):
    '''Input: a, b - longitude and latitude in radians
    Output: an orthonormal frame at the vector v
    '''
    v = np.array([[np.cos(b)*np.cos(a)],[np.cos(b)*np.sin(a)],[np.sin(b)]])
    v1 = np.array([[-np.sin(a)],[np.cos(a)],[0]])
    v2 = np.array([[-np.sin(b)*np.cos(a)], [-np.sin(b)*np.sin(a)],[np.cos(b)]])
    return v,v1,v2

convertToRadian = lambda x:list(map(lambda x1:x1*np.pi/180,x))

def greatCirclePath(lon1,lat1,lon2,lat2, pts = 50):
    '''
    Input: longitudes and latitudes of the two cities
    Output: a great circle path from one city to the other.
    '''
    a1,b1 = convertToRadian((lon1, lat1))
    a2,b2 = convertToRadian((lon2, lat2))
    v1 = np.array([[np.cos(b1)*np.cos(a1)], [np.cos(b1)*np.sin(a1)], [np.sin(b1)]])
    v2 = np.array([[np.cos(b2)*np.cos(a2)], [np.cos(b2)*np.sin(a2)], [np.sin(b2)]])
    angle = np.arccos(np.sum(v1*v2))
    angles1 = np.linspace(0,angle,pts)
    ratio2 = np.sin(angles1)/np.sin(angle)
    ratio1 = np.cos(angles1) - np.cos(angle) * ratio2
    v3s = v1 *ratio1 + v2*ratio2
    
    return v3s

def findBearings(lon1,lat1,lon2,lat2, pts = 50):
    '''
    Input: longitudes and latitudes of the two cities
    Output: bearings along the path from one city to the other.
    '''
    a1,b1 = convertToRadian((lon1, lat1))
    a2,b2 = convertToRadian((lon2, lat2))
    v1 = np.array([[np.cos(b1)*np.cos(a1)], [np.cos(b1)*np.sin(a1)], [np.sin(b1)]])
    v2 = np.array([[np.cos(b2)*np.cos(a2)], [np.cos(b2)*np.sin(a2)], [np.sin(b2)]])
    angle = np.arccos(np.sum(v1*v2))
    angles1 = np.linspace(0,angle,pts)
    ratio2 = np.sin(angles1)/np.sin(angle)
    ratio1 = np.cos(angles1) - np.cos(angle) * ratio2
    v3s = v1 *ratio1 + v2*ratio2
    
    r2 = np.cos(angles1)/np.sin(angle)
    r1 = -np.sin(angles1) - np.cos(angle) * r2
    v3s_prime = v1 *r1 + v2*r2
    xydist = np.sqrt(np.sum(v3s[:2,:]**2,axis=0,keepdims=True))
    v3s_east = v3s[[1,0,2],:] * np.array([[-1],[1],[0]])
    v3s_east = v3s_east / xydist
    v3s_north = v3s*(-v3s[2:,:])
    v3s_north[:2,:] = v3s_north[:2,:] / xydist
    v3s_north[2,:]  = xydist
    cos_east = np.sum(v3s_prime*v3s_east,axis=0)
    bearings = np.arccos(np.sum(v3s_prime*v3s_north,axis=0))
    bearings[cos_east < 0] = 2*np.pi - bearings[cos_east < 0]
    bearings = bearings*180/np.pi
    return bearings

def convertToLonLat(a):
    '''
    Input: a: a (2,n)-array of points on Earth
    Output: lons, lats: longitudes and Latitudes of points
    '''
    lons = np.arccos(a[0,:]/np.sqrt(a[0,:]**2 + a[1,:]**2))
    lons[a[1,:]<0] = -lons[a[1,:]<0]
    lats = np.arctan(a[2,:]/np.sqrt(a[0,:]**2 + a[1,:]**2))
    lons *= 180/np.pi
    lats *= 180/np.pi
    return lons, lats

def circleLonLat(lon1,lat1,angle):
    '''
    Input: long and Lat of center of circle, and radius of circle in degrees
    Output: coordinates of circumference of circle
    '''
    a1, b1 = convertToRadian((lon1, lat1))
    theta = np.linspace(0,2*np.pi,200)
    v,v1,v2 = orthonormalFrame(a1, b1)
    angle_rad = angle*np.pi/180
    alpha = v * np.cos(angle_rad) + np.sin(angle_rad)*(v1*np.cos(theta) + v2*np.sin(theta))
    return convertToLonLat(alpha)
def angleDist(lon1,lat1,lon2,lat2):
    a1, b1 = lat1*np.pi/180, lon1*np.pi/180
    a2, b2 = lat2*np.pi/180, lon2*np.pi/180
    v1 = np.array([[np.cos(a1)*np.cos(b1)], [np.cos(a1)*np.sin(b1)], [np.sin(a1)]])
    v2 = np.array([[np.cos(a2)*np.cos(b2)], [np.cos(a2)*np.sin(b2)], [np.sin(a2)]])
    angle = np.arccos(np.sum(v1*v2)) / np.pi * 180
    return angle

def dist(lon1,lat1,lon2,lat2):
    angle = angleDist(lon1,lat1,lon2,lat2) * np.pi/180
    return radius * angle

radius = 6371

def pseudoDist(lon1,lat1,lon2,lat2):
    
    a1 = np.linspace(lon1,lon2, 201)
    a2 = np.linspace(lat1,lat2, 201)
    tot = 0
    for i in range(200):
        tot += dist(a1[i],a2[i],a1[i+1],a2[i+1])
    
    if lon2 < lon1:
        lon3 = lon2 + 360
    else:
        lon3 = lon2 - 360
    b1 = np.linspace(lon1,lon3, 201)
    b2 = np.linspace(lat1,lat2, 201)
    tot1 = 0
    for i in range(200):
        tot1 += dist(b1[i],b2[i],b1[i+1],b2[i+1])
    return min(tot,tot1)

def drawMap(city1, city2, n_circle):
  if city1 is None or city2 is None:
      return
  elif city1 == city2:
    raise ValueError('Please select two different cities')

  lon1, lat1 = cities[city1]
  lon2, lat2 = cities[city2]

  
  longs, lats = convertToLonLat(greatCirclePath(lon1,lat1, lon2,lat2,50))


  plt.figure(figsize=(20,16))
  ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=160))

  ax.set_global()
  # ax.stock_img()
  ax.coastlines()
  ax.add_feature(cf.BORDERS)

  plt.scatter([lon2, lon1], [lat2, lat1],  color='blue', linewidth=2,
          transform=ccrs.Geodetic(),marker='o',)
  plt.plot(longs, lats,
          color='blue', linewidth=3,
          transform=ccrs.Geodetic(),
          )

  if lon1 - 180 < lon2 < lon1 + 180:
    plt.plot(np.linspace(lon1, lon2,20),np.linspace(lat1, lat2,20), \
      color='gray', linestyle='--', \
      transform=ccrs.PlateCarree(),
          )
  elif lon2 < lon1:
    plt.plot(np.linspace(lon1, lon2+360,20),np.linspace(lat1, lat2,20), \
        color='gray', linestyle='--', \
        transform=ccrs.PlateCarree(),
          )
  else:
    plt.plot(np.linspace(lon1+360, lon2,20),np.linspace(lat1, lat2,20), \
        color='gray', linestyle='--', \
        transform=ccrs.PlateCarree(),
          )
  plt.text(lon1 + 3, lat1 - 7, city1,
          horizontalalignment='right',
          transform=ccrs.Geodetic())

  plt.text(lon2 + 3, lat2 - 7, city2,
          horizontalalignment='left',
          transform=ccrs.Geodetic())

  angle_deg = angleDist(lon1,lat1,lon2,lat2)
  real_dist = dist(lon1,lat1,lon2,lat2)
  fake_dist = pseudoDist(lon1,lat1,lon2,lat2)

  for i in range(1,n_circle + 1):
      lon1s,lat1s = circleLonLat(lon1,lat1,angle_deg * i/n_circle)
      plt.plot(lon1s,lat1s,color='green',transform=ccrs.Geodetic())
      lon1s,lat1s = circleLonLat(lon2,lat2,angle_deg * i/n_circle)
      plt.plot(lon1s,lat1s,color='magenta',transform=ccrs.Geodetic())
  plt.title(f'{city1} - {city2}: Great circle distance: {real_dist:.1f} km, "straight line" distance: {fake_dist:.1f} km')
  # plt.show()

def drawEarth(city1, city2, n_circle):
  if city1 is None or city2 is None:
      return
  elif city1 == city2:
    raise ValueError('Please select two different cities')

  lon1, lat1 = cities[city1]
  lon2, lat2 = cities[city2]

  
  longs, lats = convertToLonLat(greatCirclePath(lon1,lat1, lon2,lat2,50))

  plt.figure(figsize=(20,16))
  ax = plt.axes(projection=ccrs.Orthographic(central_longitude = longs[24], central_latitude=max(lats[24]-20,-80)))

  ax.set_global()
  # ax.stock_img()
  ax.coastlines()
  ax.add_feature(cf.BORDERS)

  plt.scatter([lon2, lon1], [lat2, lat1],  color='blue', linewidth=2,
          transform=ccrs.Geodetic(),marker='o',)
  plt.plot(longs, lats,
          color='blue', linewidth=3,
          transform=ccrs.Geodetic(),
          )

  if lon1 - 180 < lon2 < lon1 + 180:
    plt.plot(np.linspace(lon1, lon2,20),np.linspace(lat1, lat2,20), \
      color='gray', linestyle='--', \
      transform=ccrs.PlateCarree(),
          )
  elif lon2 < lon1:
    plt.plot(np.linspace(lon1, lon2+360,20),np.linspace(lat1, lat2,20), \
        color='gray', linestyle='--', \
        transform=ccrs.PlateCarree(),
          )
  else:
    plt.plot(np.linspace(lon1+360, lon2,20),np.linspace(lat1, lat2,20), \
        color='gray', linestyle='--', \
        transform=ccrs.PlateCarree(),
          )
  plt.text(lon1 + 3, lat1 - 7, city1,
          horizontalalignment='right',
          transform=ccrs.Geodetic())

  plt.text(lon2 + 3, lat2 - 7, city2,
          horizontalalignment='left',
          transform=ccrs.Geodetic())

  angle_deg = angleDist(lon1,lat1,lon2,lat2)

  for i in range(1,n_circle + 1):
      lon1s,lat1s = circleLonLat(lon1,lat1,angle_deg * i/n_circle)
      plt.plot(lon1s,lat1s,color='green',transform=ccrs.Geodetic())
      lon1s,lat1s = circleLonLat(lon2,lat2,angle_deg * i/n_circle)
      plt.plot(lon1s,lat1s,color='magenta',transform=ccrs.Geodetic())

cities= {'Hong Kong': (114.1831,22.3069),
'Beijing': (116.4075,39.904),
'Guangzhou': (113.259,23.1288),
'Hangzhou': (120.1675,30.25),
'Hohhot': (111.6629,40.8151),
'Harbin': (126.6333,45.75),
'Taipei': (121.5319,25.0478),
'Shanghai': (121.4667,31.1667),
'Xi’an': (108.9,34.2667),
'Tokyo': (139.7744,35.6839),
'Seoul': (126.99,37.56),
'Kuala Lumpur': (101.6953,3.1478),
'Singapore': (103.8,1.3),
'Manila': (120.9833,14.6),
'New Delhi': (77.209,28.6139),
'Karachi': (67.01,24.86),
'Jakarta': (106.8451,-6.2146),
'Bangkok': (100.5167,13.75),
'New York': (-73.9249,40.6943),
'Chicago': (-87.6862,41.8373),
'Los Angeles': (-118.4068,34.1139),
'Seattle': (-122.3244,47.6211),
'Vancouver': (-123.1,49.25),
'Toronto': (-79.3733,43.7417),
'Quebec City': (-71.2081,46.8139),
'Buenos Aires': (-58.3819,-34.5997),
'Rio de Janeiro': (-43.1964,-22.9083),
'Havana': (-82.3589,23.1367),
'Paris': (2.3522,48.8566),
'Madrid': (-3.7167,40.4167),
'Lisbon': (-9.139,38.708),
'Milan': (9.19,45.4669),
'Rome': (12.4828,41.8931),
'Budapest': (19.0408,47.4983),
'Athens': (23.7281,37.9842),
'Moscow': (37.6178,55.7558),
'Saint Petersburg': (30.3167,59.95),
'Istanbul': (28.9603,41.01),
'Damascus': (36.2919,33.5131),
'Dubai': (55.3094,25.2697),
'Tehran': (51.4167,35.7),
'Cairo': (31.2358,30.0444),
'Nairobi': (36.8172,-1.2864),
'Johannesburg': (28.0416,-26.2044),
'Canberra': (149.1269,-35.2931),
'Sydney': (151.2094,-33.865),
'Wellington': (174.7772,-41.2889),
'Christchurch': (172.6365,-43.5309)}


預備工作做好後，同學可以運行以下程式碼： 然後選取出發城市及抵涉城市，便可以看到最短航道這個全局問題的最優解了。

In [6]:
city1 = widgets.Dropdown(options=list(cities.keys()),value=None,description = 'City 1:')
city2 = widgets.Dropdown(options=list(cities.keys()),value=None,description = 'City 2:')
n_circle = widgets.IntSlider(min= 1, max= 10, value = 3, description ='No. of circles')
widgets.interact(drawEarth, city1=city1,city2=city2, n_circle = n_circle)


interactive(children=(Dropdown(description='City 1:', options=('Hong Kong', 'Beijing', 'Guangzhou', 'Hangzhou'…

<function __main__.drawEarth(city1, city2, n_circle)>

以經緯作座標，地圖上以城市 A 及城市 B 為中心的包圍圈分別繪以綠色線及粉紫線，藍色曲線就是理論上的最短航道，電腦計算結果的平面圖上的直線實際距離的確比大圓航線長。

In [7]:
city1 = widgets.Dropdown(options=list(cities.keys()),value=None,description = 'City 1:')
city2 = widgets.Dropdown(options=list(cities.keys()),value=None,description = 'City 2:')
n_circle = widgets.IntSlider(min= 1, max= 10, value = 3, description ='No. of circles')
widgets.interact(drawMap, city1=city1,city2=city2, n_circle = n_circle)

interactive(children=(Dropdown(description='City 1:', options=('Hong Kong', 'Beijing', 'Guangzhou', 'Hangzhou'…

<function __main__.drawMap(city1, city2, n_circle)>

有興趣的同學可以關心以下議題：

*   由於球面與平面的曲率不一樣，完美的地圖投影並不存在，各種不一樣的地圖投影可以[在這個連結找到](https://zh.wikipedia.org/zh-hk/%E5%9C%B0%E5%9B%BE%E6%8A%95%E5%BD%B1)。
*   某樂隊計畫在環球城市（ 譬如十大熱門旅遊城市） 作巡迴演出， 如何有效地安排巡迴的次序來 減少飛行里數？ 這就是有名的組合問題--[旅行推銷員問題](https://zh.wikipedia.org/zh-hk/%E6%97%85%E8%A1%8C%E6%8E%A8%E9%94%80%E5%91%98%E9%97%AE%E9%A2%98)了。

